Importation des modules nécessaires

In [81]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from pypdf import PdfReader
import ollama
import gradio as gr
import os
import json
import ollama

In [82]:
# Assurez-vous que vous avez installé et configuré correctement ollama
ollama.pull("sciphi/triplex")

{'status': 'success'}

fonction crée une invite structurée (prompt) pour le modèle "sciphi/triplex" et génère une réponse basée sur l'entité et les prédicats spécifiés.

In [83]:
def triplextract(text, entity_types, predicates):

    input_format = """Perform Named Entity Recognition (NER) and extract knowledge graph triplets from the text. NER identifies named entities of given entity types, and triple extraction identifies relationships between entities using specified predicates.

        **Entity Types:**
        {entity_types}

        **Predicates:**
        {predicates}

        **Text:**
        {text}
        """

    message = input_format.format(
                entity_types = json.dumps({"entity_types": entity_types}),
                predicates = json.dumps({"predicates": predicates}),
                text = text)

    # Pass the message as a single string
    prompt = message
    output = ollama.generate(model='sciphi/triplex', prompt=prompt)
    return output

Cette section lit un fichier PDF et extrait le texte de chaque page, le concaténant dans une seule chaîne de caractères, et passe le texte extrait à la fonction triplextract, traite la réponse JSON pour en extraire les entités et les triples

In [84]:
entity_types = ["PERSON", "DATE", "LOCATION", "ORGANIZATION", "WORK", "CONCEPT"]
predicates = ["BORN_ON", "DIED_ON", "LOCATED_IN", "WORKED_AT", "MAJOR_CONTRIBUTIONS", "DISCOVERED", "HELD_POSITIONS"]

reader = PdfReader("data/bibliographie.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"
prediction = triplextract(text, entity_types, predicates)

response_string = prediction['response'].strip('```json\n').strip()
response_string = response_string.lstrip('\n')
response_string = response_string.strip('```')
response_string = response_string.replace('```', '')
response_string = response_string.replace("json", "")
response_json = json.loads(response_string)
entities_and_triples = response_json['entities_and_triples']

Le texte extrait est écrit dans un fichier output.txt, puis chargé à l'aide de TextLoader.

In [85]:
with open('output.txt', 'w', encoding='utf-8') as f:
    f.write(text)

loader = TextLoader("./output.txt")
docs = loader.load()

Le texte est divisé en chunks sémantiques, les embeddings sont générés à l'aide de HuggingFaceEmbeddings, et ces embeddings sont stockés dans un index FAISS pour une récupération rapide.

In [86]:
# Split into chunks
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store and fill it with embeddings
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

Cette section définit le modèle LLM et le prompt pour les questions-réponses.

In [87]:
# Define llm
llm = Ollama(model="mistral")

# Define the prompt
prompt = """
    1. Use the following pieces of context to answer the question at the end.
    2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
    3. Keep the answer crisp and limited to 3,4 sentences.

    Context: {context}

    Question: {question}

    Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

une chaîne LLM est créée pour traiter les prompts, et une chaîne de combinaison de documents est définie pour intégrer le contexte des documents.

In [88]:
llm_chain = LLMChain(
                    llm=llm, 
                    prompt=QA_CHAIN_PROMPT, 
                    callbacks=None, 
                    verbose=True)

document_prompt = PromptTemplate(
        input_variables=["page_content", "source"],
        template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                    llm_chain=llm_chain,
                    document_variable_name="context",
                    document_prompt=document_prompt,
                    callbacks=None)

Cette section configure la chaîne de questions-réponses (RetrievalQA) et définit une fonction de réponse pour répondre aux questions des utilisateurs.

In [89]:
                
qa = RetrievalQA(
                combine_documents_chain=combine_documents_chain,
                verbose=True,
                retriever=retriever,
                return_source_documents=True)

def respond(question, history):
        return qa(question)["result"]

Cette section configure et lance une interface utilisateur avec Gradio, permettant aux utilisateurs d'interagir avec le chatbot.

In [90]:
gr.ChatInterface(
        respond,
        chatbot=gr.Chatbot(height=500),
        textbox=gr.Textbox(placeholder="Ask me question related to Document ", container=False, scale=7),
        title="RAG Chatbot",
        examples=["When did Isaac Newton die?", "What were the key events and influences in Einstein's early education?"],
        cache_examples=True,
        retry_btn=None,
    ).launch(share=True)

Caching examples at: 'C:\Users\DELL\Downloads\triplex\gradio_cached_examples\82'
Caching example 1/2


> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

    1. Use the following pieces of context to answer the question at the end.
    2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

    3. Keep the answer crisp and limited to 3,4 sentences.

    Context: Context:
content:Isaac Newton, born on December 25, 1642 (according to the Julian calendar then in use in England) in 
Woolsthorpe, Lincolnshire, and died on March 20, 1727, in London, is one of the most influential scientists in 
history. His major contributions to physics,  mathematics, and astronomy laid the foundations of modern science 
and profoundly transformed our understanding of the natural world. Childhood and Education  
Isaac Newton was born prematurely into a family of farmers. His father, also named Isaac Newton, d ied t